In [1]:
import cv2
import numpy as np

In [2]:
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

In [3]:
with open('coco.names', 'r') as f:
    classes = f.read().splitlines()
print(classes)#########

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [49]:
img = cv2.imread('image.jpg')
height, width, _ = img.shape

blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
#                                 normalize, decide image 416x415, we are not going to any mean subtraction, bgr image to rgb order, not going to do crop)
# for b in blob:
#     for n, img_blob in enumerate(b):
#         cv2.imshow(str(n), img_blob)

net.setInput(blob)

output_laters_names = net.getUnconnectedOutLayersNames()
layerOutputs = net.forward(output_laters_names)

boxes = [] # extract the coundary boxes
confidences = [] # store confidences
class_ids = [] # class id which present predicted class

for output in layerOutputs:
    for detection in output:
        scores = detection[5:] #used to store all the classes prediction
        # we want to identify scores
        class_id = np.argmax(scores) #find the location contains highes scores location
        confidence = scores[class_id] # assign into confidence variable. because we want to make sure high confidence
        if confidence > 0.5:
            center_x = int(detection[0]*width) #x *width
            center_y = int(detection[1]*height) #y *height
            w = int(detection[2]*width)
            h = int(detection[3]*height)
            
            # to get position of upper left corner
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            
            boxes.append([x,y,w,h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)
    
# print(len(boxes)) # how many object are being detected
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
                                        #threshhold 50%, maximum surpression
# print(indexes.flatten()) # how many boxes are redundence. in here around 12 redundence
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(len(boxes), 3))
traffic_light = []
for i in indexes.flatten(): # loop to identify object detected
    x, y, w, h = boxes[i] # extrac information back to boxes
    label = str(classes[class_ids[i]]) # extrack back to 'str' cuz here only show number
    confidence = str(round(confidences[i],2)) # extract confidences to str that show in pics
    color = colors[i] # randomly pick colors to show each of object
    cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
    ###########################################
    if label == 'traffic light':
        print('true')
        print([x,y,w,h])
        cropped_image = img[y:y+h, x:x+w]
        traffic_light.append(cropped_image)
    cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255, 255, 255), 2) #put into text

for n, traffic_img in enumerate(traffic_light):
#     cv2.imshow(str(n), traffic_img)
    
    # Convert the imageFrame in 
    # BGR(RGB color space) to 
    # HSV(hue-saturation-value)
    # color space
    hsvFrame = cv2.cvtColor(traffic_img, cv2.COLOR_BGR2HSV)
  
    # Set range for red color and 
    # define mask
    red_lower = np.array([136, 87, 111], np.uint8)
    red_upper = np.array([180, 255, 255], np.uint8)
    red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)
  
    # Set range for green color and 
    # define mask
    green_lower = np.array([25, 52, 72], np.uint8)
    green_upper = np.array([102, 255, 255], np.uint8)
    green_mask = cv2.inRange(hsvFrame, green_lower, green_upper)
  
    # Set range for blue color and
    # define mask
    blue_lower = np.array([94, 80, 2], np.uint8)
    blue_upper = np.array([120, 255, 255], np.uint8)
    blue_mask = cv2.inRange(hsvFrame, blue_lower, blue_upper)
      
    # Morphological Transform, Dilation
    # for each color and bitwise_and operator
    # between imageFrame and mask determines
    # to detect only that particular color
    kernal = np.ones((5, 5), "uint8")
      
    # For red color
    red_mask = cv2.dilate(red_mask, kernal)
    res_red = cv2.bitwise_and(traffic_img, traffic_img, 
                              mask = red_mask)
      
    # For green color
    green_mask = cv2.dilate(green_mask, kernal)
    res_green = cv2.bitwise_and(traffic_img, traffic_img,
                                mask = green_mask)
      
#     # For blue color
#     blue_mask = cv2.dilate(blue_mask, kernal)
#     res_blue = cv2.bitwise_and(imageFrame, imageFrame,
#                                mask = blue_mask)
   
    # Creating contour to track red color
    contours, hierarchy = cv2.findContours(red_mask,
                                           cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)
      
    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > 300):
            x, y, w, h = cv2.boundingRect(contour)
            traffic_img = cv2.rectangle(traffic_img, (x, y), 
                                       (x + w, y + h), 
                                       (0, 0, 255), 2)
              
            cv2.putText(traffic_img, "Red Colour", (x, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 1.0,
                        (0, 0, 255))    
  
    # Creating contour to track green color
    contours, hierarchy = cv2.findContours(green_mask,
                                           cv2.RETR_TREE,
                                           cv2.CHAIN_APPROX_SIMPLE)
      
    for pic, contour in enumerate(contours):
        area = cv2.contourArea(contour)
        if(area > 300):
            x, y, w, h = cv2.boundingRect(contour)
            traffic_img = cv2.rectangle(traffic_img, (x, y), 
                                       (x + w, y + h),
                                       (0, 255, 0), 2)
              
            cv2.putText(traffic_img, "Green Colour", (x, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        1.0, (0, 255, 0))
    cv2.imshow(str(n), traffic_img)
  
  # Creating contour to track blue color
#     contours, hierarchy = cv2.findContours(blue_mask,
#                                            cv2.RETR_TREE,
#                                            cv2.CHAIN_APPROX_SIMPLE)
#     for pic, contour in enumerate(contours):
#         area = cv2.contourArea(contour)
#         if(area > 300):
#             x, y, w, h = cv2.boundingRect(contour)
#             imageFrame = cv2.rectangle(imageFrame, (x, y),
#                                        (x + w, y + h),
#                                        (255, 0, 0), 2)
              
#             cv2.putText(imageFrame, "Blue Colour", (x, y),
#                         cv2.FONT_HERSHEY_SIMPLEX,
#                         1.0, (255, 0, 0))

cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
#save matrix/array as image file
isWritten = cv2.imwrite('image-2.jpg', img)

if isWritten:
    print('Image is successfully saved as file.')

true
[381, 1547, 48, 87]
true
[1927, 1236, 67, 146]
true
[102, 1541, 60, 80]
Image is successfully saved as file.


In [10]:
cropped_image = img[Y:Y+H, X:X+W]
print([X,Y,W,H])
plt.imshow(cropped_image)
cv2.imwrite('contour1.png', cropped_image)

NameError: name 'Y' is not defined

In [78]:
img = cv2.imread('image.jpg')
height, width, _ = img.shape

blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
#                                 normalize, decide image 416x415, we are not going to any mean subtraction, bgr image to rgb order, not going to do crop)
# for b in blob:
#     for n, img_blob in enumerate(b):
#         cv2.imshow(str(n), img_blob)

net.setInput(blob)

output_laters_names = net.getUnconnectedOutLayersNames()
layerOutputs = net.forward(output_laters_names)

boxes = [] # extract the coundary boxes
confidences = [] # store confidences
class_ids = [] # class id which present predicted class

for output in layerOutputs:
    for detection in output:
        scores = detection[5:] #used to store all the classes prediction
        # we want to identify scores
        class_id = np.argmax(scores) #find the location contains highes scores location
        confidence = scores[class_id] # assign into confidence variable. because we want to make sure high confidence
        if confidence > 0.5:
            center_x = int(detection[0]*width) #x *width
            center_y = int(detection[1]*height) #y *height
            w = int(detection[2]*width)
            h = int(detection[3]*height)
            
            # to get position of upper left corner
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            
            boxes.append([x,y,w,h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)
    
# print(len(boxes)) # how many object are being detected
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
                                        #threshhold 50%, maximum surpression
# print(indexes.flatten()) # how many boxes are redundence. in here around 12 redundence
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(len(boxes), 3))
traffic_light = []
for i in indexes.flatten(): # loop to identify object detected
    x, y, w, h = boxes[i] # extrac information back to boxes
    label = str(classes[class_ids[i]]) # extrack back to 'str' cuz here only show number
    confidence = str(round(confidences[i],2)) # extract confidences to str that show in pics
    color = colors[i] # randomly pick colors to show each of object
#     cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
    ###########################################
    if label == 'traffic light':
        print('true')
        print([x,y,w,h])
        traffic_img = img[y:y+h, x:x+w]
#         cropped_image = img[y:y+h, x:x+w]

#         # collect traffic_light
#         traffic_light.append(cropped_image)
        # Convert the image in BGR to HSV(hue-saturation-value) color space
        hsvFrame = cv2.cvtColor(traffic_img, cv2.COLOR_BGR2HSV)
  
        # Set range for red color and 
        # define mask
        red_lower = np.array([136, 87, 111], np.uint8)
        red_upper = np.array([180, 255, 255], np.uint8)
        red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)

        # Set range for green color and 
        # define mask
        green_lower = np.array([25, 62, 82], np.uint8)
        green_upper = np.array([102, 255, 255], np.uint8)
        green_mask = cv2.inRange(hsvFrame, green_lower, green_upper)

#         # Set range for blue color and
#         # define mask
#         blue_lower = np.array([94, 80, 2], np.uint8)
#         blue_upper = np.array([120, 255, 255], np.uint8)
#         blue_mask = cv2.inRange(hsvFrame, blue_lower, blue_upper)

        # Morphological Transform, Dilation
        # for each color and bitwise_and operator
        # between imageFrame and mask determines
        # to detect only that particular color
        kernal = np.ones((5, 5), "uint8")

        # For red color
        red_mask = cv2.dilate(red_mask, kernal)
        res_red = cv2.bitwise_and(traffic_img, traffic_img, 
                                  mask = red_mask)

        # For green color
        green_mask = cv2.dilate(green_mask, kernal)
        res_green = cv2.bitwise_and(traffic_img, traffic_img,
                                    mask = green_mask)

    #     # For blue color
    #     blue_mask = cv2.dilate(blue_mask, kernal)
    #     res_blue = cv2.bitwise_and(imageFrame, imageFrame,
    #                                mask = blue_mask)

        # Creating contour to track red color
        contours, hierarchy = cv2.findContours(red_mask,
                                               cv2.RETR_TREE,
                                               cv2.CHAIN_APPROX_SIMPLE)
#         print('a1')
#         print(hierarchy)
        rg_eixit = False
        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if(area > 300):
                xt, yt, wt, ht = cv2.boundingRect(contour)
                traffic_img = cv2.rectangle(traffic_img, (xt, yt), 
                                           (xt + wt, yt + ht), 
                                           (0, 0, 255), 2)

#                 cv2.putText(traffic_img, "Red Colour", (xt, yt),
#                             cv2.FONT_HERSHEY_SIMPLEX, 1.0,
#                             (0, 0, 255))    
                cv2.putText(img, "RED:"+label+confidence, (x, y), font, 2, (0, 0, 255), 2) #put into text
                rg_eixit = True
#                 break

        # Creating contour to track green color
        contours, hierarchy = cv2.findContours(green_mask,
                                               cv2.RETR_TREE,
                                               cv2.CHAIN_APPROX_SIMPLE)
#         print('a2')
#         print(hierarchy)

        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if(area > 300):
                xt, yt, wt, ht = cv2.boundingRect(contour)
                traffic_img = cv2.rectangle(traffic_img, (xt, yt), 
                                           (xt + wt, yt + ht),
                                           (0, 255, 0), 2)

#                 cv2.putText(traffic_img, "Green Colour", (x, y),
#                             cv2.FONT_HERSHEY_SIMPLEX, 
#                             1.0, (0, 255, 0))
                cv2.putText(img, "GREEN:"+label + confidence, (x, y), font, 2, (0, 255, 0), 2) #put into text
                rg_eixit = True
    #                 break
#         cv2.imshow(str(n), traffic_img)



# #############################################################
#     else:
        if not rg_eixit:
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
            cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255, 255, 255), 2) #put into text

    else:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
        cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255, 255, 255), 2) #put into text

# for n, traffic_img in enumerate(traffic_light):
# #     cv2.imshow(str(n), traffic_img)
    
#     # Convert the imageFrame in 
#     # BGR(RGB color space) to 
#     # HSV(hue-saturation-value)
#     # color space
#     hsvFrame = cv2.cvtColor(traffic_img, cv2.COLOR_BGR2HSV)
  
#     # Set range for red color and 
#     # define mask
#     red_lower = np.array([136, 87, 111], np.uint8)
#     red_upper = np.array([180, 255, 255], np.uint8)
#     red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)
  
#     # Set range for green color and 
#     # define mask
#     green_lower = np.array([25, 52, 72], np.uint8)
#     green_upper = np.array([102, 255, 255], np.uint8)
#     green_mask = cv2.inRange(hsvFrame, green_lower, green_upper)
  
#     # Set range for blue color and
#     # define mask
#     blue_lower = np.array([94, 80, 2], np.uint8)
#     blue_upper = np.array([120, 255, 255], np.uint8)
#     blue_mask = cv2.inRange(hsvFrame, blue_lower, blue_upper)
      
#     # Morphological Transform, Dilation
#     # for each color and bitwise_and operator
#     # between imageFrame and mask determines
#     # to detect only that particular color
#     kernal = np.ones((5, 5), "uint8")
      
#     # For red color
#     red_mask = cv2.dilate(red_mask, kernal)
#     res_red = cv2.bitwise_and(traffic_img, traffic_img, 
#                               mask = red_mask)
      
#     # For green color
#     green_mask = cv2.dilate(green_mask, kernal)
#     res_green = cv2.bitwise_and(traffic_img, traffic_img,
#                                 mask = green_mask)
      
# #     # For blue color
# #     blue_mask = cv2.dilate(blue_mask, kernal)
# #     res_blue = cv2.bitwise_and(imageFrame, imageFrame,
# #                                mask = blue_mask)
   
#     # Creating contour to track red color
#     contours, hierarchy = cv2.findContours(red_mask,
#                                            cv2.RETR_TREE,
#                                            cv2.CHAIN_APPROX_SIMPLE)
      
#     for pic, contour in enumerate(contours):
#         area = cv2.contourArea(contour)
#         if(area > 300):
#             x, y, w, h = cv2.boundingRect(contour)
#             traffic_img = cv2.rectangle(traffic_img, (x, y), 
#                                        (x + w, y + h), 
#                                        (0, 0, 255), 2)
              
#             cv2.putText(traffic_img, "Red Colour", (x, y),
#                         cv2.FONT_HERSHEY_SIMPLEX, 1.0,
#                         (0, 0, 255))    
  
#     # Creating contour to track green color
#     contours, hierarchy = cv2.findContours(green_mask,
#                                            cv2.RETR_TREE,
#                                            cv2.CHAIN_APPROX_SIMPLE)
      
#     for pic, contour in enumerate(contours):
#         area = cv2.contourArea(contour)
#         if(area > 300):
#             x, y, w, h = cv2.boundingRect(contour)
#             traffic_img = cv2.rectangle(traffic_img, (x, y), 
#                                        (x + w, y + h),
#                                        (0, 255, 0), 2)
              
#             cv2.putText(traffic_img, "Green Colour", (x, y),
#                         cv2.FONT_HERSHEY_SIMPLEX, 
#                         1.0, (0, 255, 0))
#     cv2.imshow(str(n), traffic_img)
  
#   # Creating contour to track blue color
# #     contours, hierarchy = cv2.findContours(blue_mask,
# #                                            cv2.RETR_TREE,
# #                                            cv2.CHAIN_APPROX_SIMPLE)
# #     for pic, contour in enumerate(contours):
# #         area = cv2.contourArea(contour)
# #         if(area > 300):
# #             x, y, w, h = cv2.boundingRect(contour)
# #             imageFrame = cv2.rectangle(imageFrame, (x, y),
# #                                        (x + w, y + h),
# #                                        (255, 0, 0), 2)
              
# #             cv2.putText(imageFrame, "Blue Colour", (x, y),
# #                         cv2.FONT_HERSHEY_SIMPLEX,
# #                         1.0, (255, 0, 0))

cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
#save matrix/array as image file
isWritten = cv2.imwrite('image-2.jpg', img)

if isWritten:
    print('Image is successfully saved as file.')

true
[381, 1547, 48, 87]
true
[1927, 1236, 67, 146]
true
[102, 1541, 60, 80]
Image is successfully saved as file.


In [4]:
def intersection_over_union(boxA, boxB):
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA + 1) * max(0, yB - yA + 1)
    # compute the area of both the prediction and ground-truth
    # rectangles
    boxAArea = (boxA[2] - boxA[0] + 1) * (boxA[3] - boxA[1] + 1)
    boxBArea = (boxB[2] - boxB[0] + 1) * (boxB[3] - boxB[1] + 1)
    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = interArea / float(boxAArea + boxBArea - interArea)
    # return the intersection over union value
    return iou

In [150]:
# img = cv2.imread('image6.jpeg')
img = cv2.imread('image3.jpg')

height, width, _ = img.shape
center_y, center_x = height/2, width/2
height_range, width_range = height/7, width/7
box1 = [center_x-width_range, center_y-height_range, center_x+width_range, center_y+height_range]

blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
#                                 normalize, decide image 416x416, we are not going to any mean subtraction, bgr image to rgb order, not going to do crop)
# for b in blob:
#     for n, img_blob in enumerate(b):
#         cv2.imshow(str(n), img_blob)

net.setInput(blob)

output_laters_names = net.getUnconnectedOutLayersNames()
layerOutputs = net.forward(output_laters_names)

boxes = [] # extract the coundary boxes
confidences = [] # store confidences
class_ids = [] # class id which present predicted class
current_sig = None #RED, GREEN, None
car_speed = "increase" #increase, decrease, stop
car_iou_max = 0
for output in layerOutputs:
    for detection in output:
        scores = detection[5:] #used to store all the classes prediction
        # we want to identify scores
        class_id = np.argmax(scores) #find the location contains highes scores location
        confidence = scores[class_id] # assign into confidence variable. because we want to make sure high confidence
        if confidence > 0.5:
            center_x = int(detection[0]*width) #x *width
            center_y = int(detection[1]*height) #y *height
            w = int(detection[2]*width)
            h = int(detection[3]*height)
            
            # to get position of upper left corner
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            
            boxes.append([x,y,w,h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)
    
# print(len(boxes)) # how many object are being detected
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
                                        #threshhold 50%, maximum surpression
# print(indexes.flatten()) # how many boxes are redundence. in here around 12 redundence
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(len(boxes), 3))
traffic_light = []
for i in indexes.flatten(): # loop to identify object detected
    x, y, w, h = boxes[i] # extrac information back to boxes
    label = str(classes[class_ids[i]]) # extrack back to 'str' cuz here only show number
    confidence = str(round(confidences[i],2)) # extract confidences to str that show in pics
    color = colors[i] # randomly pick colors to show each of object
#     cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
    ###########################################
    if label == 'traffic light':
        print('true')
        print([x,y,w,h])
        traffic_img = img[y:y+h, x:x+w]
#         cropped_image = img[y:y+h, x:x+w]

#         # collect traffic_light
#         traffic_light.append(cropped_image)
        # Convert the image in BGR to HSV(hue-saturation-value) color space
        hsvFrame = cv2.cvtColor(traffic_img, cv2.COLOR_BGR2HSV)
  
        # Set range for red color and 
        # define mask
        red_lower = np.array([136, 87, 91], np.uint8)
        red_upper = np.array([180, 255, 255], np.uint8)
        red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)

        # Set range for green color and 
        # define mask
        green_lower = np.array([55, 102, 122], np.uint8)
        green_upper = np.array([102, 255, 255], np.uint8)
        green_mask = cv2.inRange(hsvFrame, green_lower, green_upper)



        # Morphological Transform, Dilation
        # for each color and bitwise_and operator
        # between imageFrame and mask determines
        # to detect only that particular color
        kernal = np.ones((5, 5), "uint8")

        # For red color
        red_mask = cv2.dilate(red_mask, kernal)
        res_red = cv2.bitwise_and(traffic_img, traffic_img, 
                                  mask = red_mask)

        # For green color
        green_mask = cv2.dilate(green_mask, kernal)
        res_green = cv2.bitwise_and(traffic_img, traffic_img,
                                    mask = green_mask)


        # Creating contour to track red color
        contours, hierarchy = cv2.findContours(red_mask,
                                               cv2.RETR_TREE,
                                               cv2.CHAIN_APPROX_SIMPLE)
        rg_eixit = False
        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if(area > 190):
                xt, yt, wt, ht = cv2.boundingRect(contour)
                traffic_img = cv2.rectangle(traffic_img, (xt, yt), 
                                           (xt + wt, yt + ht), 
                                           (0, 0, 255), 2)

#                 cv2.putText(traffic_img, "Red Colour", (xt, yt),
#                             cv2.FONT_HERSHEY_SIMPLEX, 1.0,
#                             (0, 0, 255))    
                cv2.putText(img, "RED:"+label+confidence, (x, y), font, 2, (0, 0, 255), 2) #put into text
                rg_eixit = True
                current_sig = "RED"
#                 break

        # Creating contour to track green color
        contours, hierarchy = cv2.findContours(green_mask,
                                               cv2.RETR_TREE,
                                               cv2.CHAIN_APPROX_SIMPLE)

        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if(area > 300):
                xt, yt, wt, ht = cv2.boundingRect(contour)
                traffic_img = cv2.rectangle(traffic_img, (xt, yt), 
                                           (xt + wt, yt + ht),
                                           (0, 255, 0), 2)

#                 cv2.putText(traffic_img, "Green Colour", (x, y),
#                             cv2.FONT_HERSHEY_SIMPLEX, 
#                             1.0, (0, 255, 0))
                cv2.putText(img, "GREEN:"+label + confidence, (x, y), font, 2, (0, 255, 0), 2) #put into text
                rg_eixit = True
                current_sig = "GREEN"
    #                 break
#         cv2.imshow(str(n), traffic_img)



#     else:
        if not rg_eixit:
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
            cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255, 255, 255), 2) #put into text

    else:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
        cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255, 255, 255), 2) #put into text
        
    # #############################################################
    if label == 'car':
        print('car')
        iou = intersection_over_union(box1, [x,y,w,h])
#         iou = intersection_over_union([x,y,w,h],box1)
        car_iou_max = max(iou, car_iou_max)
        print(iou)
#         print('car true')
        print([x,y,w,h])
        if car_iou_max > 0.2:
            car_speed = 'stop'
        elif 0.2 >= car_iou_max >0.01:
            car_speed = 'decrease'
        else:
            car_speed = 'increase'
#         print(w*h)
#         print(height*width)
#         print('car percentage: ')
#         print((w*h)/(height*width))
#         if (w*h)/(height*width) > 0.40:
#             car_speed = 'stop'
#         elif 0.40 >= (w*h)/(height*width) > 0.30:
#             car_speed = 'decrease'
#         else:
#             car_speed = 'increase'
            
#         car_img = img[y:y+h, x:x+w]

print(current_sig)
print(car_speed)
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
#save matrix/array as image file
isWritten = cv2.imwrite('image-3.jpg', img)

if isWritten:
    print('Image is successfully saved as file.')

car
0.36545720331926407
[238, 281, 742, 377]
true
[761, 134, 63, 114]
true
[44, 125, 73, 116]
car
0.0
[888, 393, 48, 50]
RED
stop
Image is successfully saved as file.


In [7]:
# img = cv2.imread('image.jpg')
img = cv2.imread('shoes4.jpg')

height, width, _ = img.shape
center_y, center_x = height/2, width/2
height_range, width_range = height/7, width/7
box1 = [center_x-width_range, center_y-height_range, center_x+width_range, center_y+height_range]

blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)
#                                 normalize, decide image 416x416, we are not going to any mean subtraction, bgr image to rgb order, not going to do crop)
# for b in blob:
#     for n, img_blob in enumerate(b):
#         cv2.imshow(str(n), img_blob)

net.setInput(blob)

output_laters_names = net.getUnconnectedOutLayersNames()
layerOutputs = net.forward(output_laters_names)

boxes = [] # extract the coundary boxes
confidences = [] # store confidences
class_ids = [] # class id which present predicted class
current_sig = None #RED, GREEN, None
car_speed = "increase" #increase, decrease, stop
car_iou_max = 0
stop_sign = 'None'
for output in layerOutputs:
    for detection in output:
        scores = detection[5:] #used to store all the classes prediction
        # we want to identify scores
        class_id = np.argmax(scores) #find the location contains highes scores location
        confidence = scores[class_id] # assign into confidence variable. because we want to make sure high confidence
        if confidence > 0.5:
            center_x = int(detection[0]*width) #x *width
            center_y = int(detection[1]*height) #y *height
            w = int(detection[2]*width)
            h = int(detection[3]*height)
            
            # to get position of upper left corner
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            
            boxes.append([x,y,w,h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)
    
# print(len(boxes)) # how many object are being detected
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
                                        #threshhold 50%, maximum surpression
# print(indexes.flatten()) # how many boxes are redundence. in here around 12 redundence
font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(len(boxes), 3))
traffic_light = []
for i in indexes.flatten(): # loop to identify object detected
    x, y, w, h = boxes[i] # extrac information back to boxes
    label = str(classes[class_ids[i]]) # extrack back to 'str' cuz here only show number
    confidence = str(round(confidences[i],2)) # extract confidences to str that show in pics
    color = colors[i] # randomly pick colors to show each of object
#     cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
    condition = ''
    ###########################################
    if label == 'traffic light':
#         print('true')
#         print([x,y,w,h])
        traffic_img = img[y:y+h, x:x+w]
#         cropped_image = img[y:y+h, x:x+w]

#         # collect traffic_light
#         traffic_light.append(cropped_image)
        # Convert the image in BGR to HSV(hue-saturation-value) color space
        hsvFrame = cv2.cvtColor(traffic_img, cv2.COLOR_BGR2HSV)
  
        # Set range for red color and 
        # define mask
        red_lower = np.array([136, 87, 91], np.uint8)
        red_upper = np.array([180, 255, 255], np.uint8)
        red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)

        # Set range for green color and 
        # define mask
        green_lower = np.array([55, 102, 122], np.uint8)
        green_upper = np.array([102, 255, 255], np.uint8)
        green_mask = cv2.inRange(hsvFrame, green_lower, green_upper)



        # Morphological Transform, Dilation
        # for each color and bitwise_and operator
        # between imageFrame and mask determines
        # to detect only that particular color
        kernal = np.ones((5, 5), "uint8")

        # For red color
        red_mask = cv2.dilate(red_mask, kernal)
        res_red = cv2.bitwise_and(traffic_img, traffic_img, 
                                  mask = red_mask)

        # For green color
        green_mask = cv2.dilate(green_mask, kernal)
        res_green = cv2.bitwise_and(traffic_img, traffic_img,
                                    mask = green_mask)


        # Creating contour to track red color
        contours, hierarchy = cv2.findContours(red_mask,
                                               cv2.RETR_TREE,
                                               cv2.CHAIN_APPROX_SIMPLE)
        rg_eixit = False
        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if(area > 190):
                xt, yt, wt, ht = cv2.boundingRect(contour)
                traffic_img = cv2.rectangle(traffic_img, (xt, yt), 
                                           (xt + wt, yt + ht), 
                                           (0, 0, 255), 2)


                cv2.putText(img, "RED:"+label+confidence, (x, y), font, 2, (0, 0, 255), 2) #put into text
                rg_eixit = True
                current_sig = "RED"

        # Creating contour to track green color
        contours, hierarchy = cv2.findContours(green_mask,
                                               cv2.RETR_TREE,
                                               cv2.CHAIN_APPROX_SIMPLE)

        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if(area > 300):
                xt, yt, wt, ht = cv2.boundingRect(contour)
                traffic_img = cv2.rectangle(traffic_img, (xt, yt), 
                                           (xt + wt, yt + ht),
                                           (0, 255, 0), 2)


                cv2.putText(img, "GREEN:"+label + confidence, (x, y), font, 2, (0, 255, 0), 2) #put into text
                rg_eixit = True
                current_sig = "GREEN"

        if not rg_eixit:
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
            cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255, 255, 255), 2) #put into text

    else:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
        cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255, 255, 255), 2) #put into text
        
    # #############################################################
    if label in ['person', 'bicycle', 'car', 'motorbike', 'bus', 'train', 'truck']:
#         print('car')
        iou = intersection_over_union(box1, [x,y,w,h])
#         iou = intersection_over_union([x,y,w,h],box1)
        car_iou_max = max(iou, car_iou_max)
        if iou != 0 and car_iou_max == iou:
#         print(iou)
#         print('car true')
#         print([x,y,w,h])
            if car_iou_max > 0.18:
                car_speed = 'stop'
            elif 0.18 >= car_iou_max >0.01:
                car_speed = 'decrease'
            else:
                car_speed = 'increase'
            condition = car_speed

            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
            cv2.putText(img, 'Car status: '+condition,(x, y+50), font, 2, (100, 70, 200), 2) #put into text
    
    if label == 'stop sign':
        stop_sign = 'stop sign'
        
print(current_sig)
print(car_speed)
print(stop_sign)
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
#save matrix/array as image file
isWritten = cv2.imwrite('image-3.jpg', img)

if isWritten:
    print('Image is successfully saved as file.')

None
decrease
None
Image is successfully saved as file.


In [2]:
import cv2
import numpy as np

net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')

with open('coco.names', 'r') as f:
    classes = f.read().splitlines()
print(classes)

['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']


In [8]:
img = cv2.imread('image3.jpg')

height, width, _ = img.shape
center_y, center_x = height/2, width/2
height_range, width_range = height/7, width/7
box1 = [center_x-width_range, center_y-height_range, center_x+width_range, center_y+height_range]

blob = cv2.dnn.blobFromImage(img, 1/255, (416, 416), (0,0,0), swapRB=True, crop=False)

net.setInput(blob)

output_laters_names = net.getUnconnectedOutLayersNames()
layerOutputs = net.forward(output_laters_names)

boxes = [] 
confidences = [] 
class_ids = [] 
current_sig = None 
car_speed = "increase" 
car_iou_max = 0
stop_sign = 'None'
for output in layerOutputs:
    for detection in output:
        scores = detection[5:] #used to store all the classes prediction
        # we want to identify scores
        class_id = np.argmax(scores) 
        confidence = scores[class_id] # assign into confidence variable. 
        if confidence > 0.5:
            center_x = int(detection[0]*width) #x *width
            center_y = int(detection[1]*height) #y *height
            w = int(detection[2]*width)
            h = int(detection[3]*height)
            
            # to get position of upper left corner
            x = int(center_x - w/2)
            y = int(center_y - h/2)
            
            boxes.append([x,y,w,h])
            confidences.append((float(confidence)))
            class_ids.append(class_id)
    
indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

font = cv2.FONT_HERSHEY_PLAIN
colors = np.random.uniform(0, 255, size=(len(boxes), 3))
traffic_light = []
for i in indexes.flatten(): # loop to identify object detected
    x, y, w, h = boxes[i] # extrac information back to boxes
    label = str(classes[class_ids[i]]) # extrack back to 'str' cuz here only show number
    confidence = str(round(confidences[i],2)) # extract confidences to str that show in pics
    color = colors[i] # randomly pick colors to show each of object
    condition = ''
    
    if label == 'traffic light':
        traffic_img = img[y:y+h, x:x+w]
        # Convert the image in BGR to HSV(hue-saturation-value) color space
        hsvFrame = cv2.cvtColor(traffic_img, cv2.COLOR_BGR2HSV)
  
        # Set range for red color and 
        # define mask
        red_lower = np.array([136, 87, 91], np.uint8)
        red_upper = np.array([180, 255, 255], np.uint8)
        red_mask = cv2.inRange(hsvFrame, red_lower, red_upper)

        # Set range for green color and 
        # define mask
        green_lower = np.array([55, 102, 122], np.uint8)
        green_upper = np.array([102, 255, 255], np.uint8)
        green_mask = cv2.inRange(hsvFrame, green_lower, green_upper)



        # Morphological Transform, Dilation
        # for each color and bitwise_and operator
        # between imageFrame and mask determines
        # to detect only that particular color
        kernal = np.ones((5, 5), "uint8")

        # For red color
        red_mask = cv2.dilate(red_mask, kernal)
        res_red = cv2.bitwise_and(traffic_img, traffic_img, 
                                  mask = red_mask)

        # For green color
        green_mask = cv2.dilate(green_mask, kernal)
        res_green = cv2.bitwise_and(traffic_img, traffic_img,
                                    mask = green_mask)


        # Creating contour to track red color
        contours, hierarchy = cv2.findContours(red_mask,
                                               cv2.RETR_TREE,
                                               cv2.CHAIN_APPROX_SIMPLE)
        rg_eixit = False
        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if(area > 190):
                xt, yt, wt, ht = cv2.boundingRect(contour)
                traffic_img = cv2.rectangle(traffic_img, (xt, yt), 
                                           (xt + wt, yt + ht), 
                                           (0, 0, 255), 2)


                cv2.putText(img, "RED:"+label+confidence, (x, y), font, 2, (0, 0, 255), 2) #put into text
                rg_eixit = True
                current_sig = "RED"

        # Creating contour to track green color
        contours, hierarchy = cv2.findContours(green_mask,
                                               cv2.RETR_TREE,
                                               cv2.CHAIN_APPROX_SIMPLE)

        for pic, contour in enumerate(contours):
            area = cv2.contourArea(contour)
            if(area > 300):
                xt, yt, wt, ht = cv2.boundingRect(contour)
                traffic_img = cv2.rectangle(traffic_img, (xt, yt), 
                                           (xt + wt, yt + ht),
                                           (0, 255, 0), 2)


                cv2.putText(img, "GREEN:"+label + confidence, (x, y), font, 2, (0, 255, 0), 2) #put into text
                rg_eixit = True
                current_sig = "GREEN"

        if not rg_eixit:
            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
            cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255, 255, 255), 2) #put into text

    else:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
        cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255, 255, 255), 2) #put into text
        
    if label in ['person', 'bicycle', 'car', 'motorbike', 'bus', 'train', 'truck']:
        iou = intersection_over_union(box1, [x,y,w,h])
        car_iou_max = max(iou, car_iou_max)
        if iou != 0 and car_iou_max == iou:

            if car_iou_max > 0.18:
                car_speed = 'stop'
            elif 0.18 >= car_iou_max >0.01:
                car_speed = 'decrease'
            else:
                car_speed = 'increase'
            condition = car_speed

            cv2.rectangle(img, (x,y), (x+w, y+h), color, 2) # create rectangle
            cv2.putText(img, 'Car status: '+condition,(x, y+50), font, 2, (100, 70, 200), 2) #put into text
    
    if label == 'stop sign':
        stop_sign = 'stop sign'
        
print(current_sig)
print(car_speed)
print(stop_sign)
cv2.imshow('Image', img)
cv2.waitKey(0)
cv2.destroyAllWindows()
#save matrix/array as image file
isWritten = cv2.imwrite('image-3.jpg', img)

if isWritten:
    print('Image is successfully saved as file.')

RED
stop
None
Image is successfully saved as file.
